In [14]:
import cyvcf2
import numpy as np

In [ ]:
# download data if needed
chr_num = 1 # which chromosome's data to download
!wget -P data/ http://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/ALL.chr1.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz

In [ ]:
variant_count = 6468094  # number of variants to use

# extract the samples and variants
variants = np.empty((variant_count,), dtype=object)

variant_index = 0  # index for the variants array

vcf_file = cyvcf2.VCF('data/ALL.chr1.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz')
samples = vcf_file.samples

for count, variant in enumerate(vcf_file):
    if count % 10000 == 0:
        print(f'Variants processed: {count}')

    variants[variant_index] = variant
    variant_index += 1

vcf_file.close()
print('Done extracting!')

In [11]:
"""
Create matrix containing a numeric representation of genotypes for each individual at each variant site
Genotype of 0: Homozygous reference genotype, both alleles at the variant site are identical to the reference allele
Genotype of 1: Heterozygous genotype, only one allele at the variant site is identical to the reference allele
Genotype of 2: Homozygous alternate genotype, both alleles at the variant site are different from the reference allele
"""
genotypes = np.zeros((len(samples), len(variants)), dtype=np.float32)
for i, variant in enumerate(variants):
    variant = str(variant).strip().split()
    for j in range(len(samples)):
        call = variant[9+j].split(':')[0]
        if call == '.|.' or call == '.|. .|.': # missing or invalid genotype at variant site
            print(f'Skipping sample {j} for variant {i} due to missing call: {call}')
            genotypes[j, i] = np.nan
        else:
            alleles = call.split('|')
            genotypes[j, i] = sum(map(int, alleles))

# filter out samples with missing/invalid genotypes at any variant site
genotypes = genotypes[~np.isnan(genotypes).any(axis=1), :]

print('Matrix created!') # takes approx 4.4hrs to process whole chromosome

Matrix created!


In [19]:
chr_num = 1

# dimensions of genotypes matrix
print('Genotypes matrix shape:', genotypes.shape)

# inspect subset of matrix values
print('Subset of matrix values:')
print(genotypes[:20, :20])

# save genotypes matrix and samples to use in pca.ipynb
np.savez_compressed(f'data/chr{chr_num}_genotypes{variant_count}.npy', genotypes)
np.save('data/samples', samples)

Genotypes matrix shape: (2504, 6468094)
Subset of matrix values:
[[1. 0. 1. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 1. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0.]
 [1. 0. 1. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 1. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0.]
 [1. 0. 1. 0. 0. 0. 0. 0. 0. 2. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 1. 0. 0. 0. 0. 0. 0. 2. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0.]
 [1. 0. 1. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 